# 二叉堆(binary heap)

In [16]:
# http://www.cnblogs.com/skywang12345/p/3610187.html

def Parent(i):
    return (i-1)>>1
def Left(i):
    return (i<<1) + 1;
def Right(i):
    return (i<<1) + 2;

class BinaryHeap:
    def __init__(self):
        self.heap = []
    
    def filter_up(self, start):
        assert start < len(self.heap)
        if(0 == start):
            return
        
        parent = Parent(start)
        if(self.heap[start] > self.heap[parent]):
            (self.heap[start], self.heap[parent]) = (self.heap[parent], self.heap[start])
            self.filter_up(parent)
    
    def filter_down(self, start, end):
        if(start > end):
            return
        pos = Left(start)
        if(pos > end):
            return
        if(pos < end and self.heap[pos] < self.heap[pos+1]):
            pos += 1
        if(self.heap[start] < self.heap[pos]):
            (self.heap[start], self.heap[pos]) = (self.heap[pos], self.heap[start])
            start = pos
            self.filter_down(start, end)
        
    def insert(self, val):
        self.heap.append(val)
        self.filter_up(len(self.heap) - 1)
    
    def remove(self, val):
        if(not val in self.heap):
            return
        idx = self.heap.index(val)
        if(1 == len(self.heap) or idx == len(self.heap) - 1):
            del(self.heap[idx])
            return
        (self.heap[idx], self.heap[len(self.heap) - 1]) = (self.heap[len(self.heap) - 1], self.heap[idx])
        self.heap.pop()
        self.filter_down(idx, len(self.heap)-1)

if __name__=='__main__':
    b_heap = BinaryHeap()
    arr = [10, 40, 30, 60, 90, 70, 20, 50, 80, 85]
    for i in arr:
        b_heap.insert(i)
    print(b_heap.heap)
    
    b_heap.remove(90)
    print(b_heap.heap)

[90, 85, 70, 60, 80, 30, 20, 10, 50, 40]
[85, 80, 70, 60, 40, 30, 20, 10, 50]


# LeftistHeap(左倾堆)

In [33]:
class LeftistNode:
    def __init__(self, key, left=None, right=None):
        self.key = key
        self.npl = 0   #Null Path Length
        self.left = left
        self.right = right
    def swap(self, node):
        (self.key, node.key) = (node.key, self.key)
        (self.npl, node.npl) = (node.npl, self.npl)
        (self.left, node.left) = (node.left, self.left)
        (self.right, node.right) = (node.right, self.right)
        
def dumpNode(node, key=None, direction=0):
    if(None != node):
        if(0 == direction):
            print("%d(%d) is root" % (node.key, node.npl))
        elif(-1 == direction):
            print("%d(%d) is %d's left child" % (node.key, node.npl, key))
        else:
            print("%d(%d) is %d's right child" % (node.key, node.npl, key))
        dumpNode(node.left, node.key, -1)
        dumpNode(node.right, node.key, 1)
        
class LeftistHeap:
    def __init__(self):
        self.root = None

    def inorder(self, arr, node):
        if(None == node):
            return
        arr.append(node)
        self.inorder(arr, node.left)
        self.inorder(arr, node.right)
        
    def dump(self):
        if(None == self.root):
            print("Tree is empty")
            return
        dumpNode(self.root)
        
    def merge(self, x_node, y_node):
        #merge a leftist heap to current heap
        if(None == x_node):
            return y_node
        if(None == y_node):
            return x_node
        if(x_node.key > y_node.key):
            x_node.swap(y_node)
        x_node.right = self.merge(x_node.right, y_node)
        if(None == x_node.left or x_node.left.npl < x_node.right.npl):
            (x_node.left, x_node.right) = (x_node.right, x_node.left)
        if(None == x_node.left or None == x_node.right):
            x_node.npl = 0
        else:
            x_node.npl = min(x_node.left.npl, x_node.right.npl) + 1
        return x_node
    
    def insert(self, val):
        node = LeftistNode(val)
        self.root = self.merge(self.root, node)

if __name__=='__main__':
    arr = [10, 40, 24, 30, 36, 20, 12, 16]
    heap = LeftistHeap()
    for i in arr:
        heap.insert(i)
    heap.dump()
    

10(2) is root
24(1) is 10's left child
30(0) is 24's left child
36(0) is 24's right child
12(1) is 10's right child
20(0) is 12's left child
40(0) is 20's left child
16(0) is 12's right child


# 二项堆(Binomial Heap)

In [5]:
# http://www.cnblogs.com/skywang12345/p/3655900.html#a1

class BinomialNode:
    def __init__(self, key, degree=0, child=None, parent=None, next=None):
        self.key = key
        self.degree = degree
        self.child = child   #child 指向二项树的最左的子树
        self.parent = parent
        self.next = next  #next指向二项堆中下一个二项树，或二项树内的兄弟节点

class BinomialHeap:
    def __init__(self):
        self.root = None
    
    def dump(self):
        if(None == self.root):
            print("Bionmial is empty")
            return
        p = self.root
        
        while(None != p):
            print "B%d " % p.degree,
            p = p.next
        print(")的详细信息:")
        p = self.root
        while(None != p):
            print("二项树B%d:" % p.degree)
            print("    %d(%d) is root" % (p.key, p.degree))
            p = p.next
    
    def search(self, root, key):
        parent = root
        while(None != parent):
            if(parent.key == key):
                return parent
            else:
                child = self.search(parent.child, key)
                if(None != child):
                    return child
                parent = parent.next
        return None
    
    def merge(self, h1, h2):
        #将h1, h2中的根表合并成一个按度数递增的链表，返回合并后的根节点
        if(None == h1):
            return h2
        if(None == h2):
            return h1
        if(h1.degree > h2.degree):
            (h1, h2) = (h2, h1)
        root = h1
        pre = h1
        h1 = h1.next
        while(None != h1 and None != h2):
            if(h1.degree < h2.degree):
                pre.next = h1
                pre = h1
                h1 = h1.next
            else:
                pre.next = h2
                pre = h2
                h2 = h2.next
        if(None != h1):
            pre.next = h1
        else:
            pre.next = h2
        return root
    
    def link(self, child, root):
        child.parent = root
        child.next = root.child
        root.child = child
        root.degree+=1

    def combine(self, h1, h2):
        root = self.merge(h1, h2)
        if(None == root):
            return None
        prev_x = None
        x = root
        next_x = x.next
        while(None != next_x):
            if(x.degree != next_x.degree or (None != next_x.next and next_x.degree == next_x.next.degree)):
                # case1: x.degree != next_x.degree
                # case2: x.degree == next_x.degree == next_x.next.degree
                prev_x = x
                x = next_x
            elif(x.key <= next_x.key):
                # x.degree == next_x.degree != next_x.next.degree
                # set next_t as x's left child
                x.next = next_x.next
                self.link(next_x, x)
            else:
                # x.degree == next_x.degree != next_x.next.degree and x.key > next_x.key
                # set x as next_t's left child
                if(None == prev_x):
                    root = next_x
                else:
                    prev_x.next = next_x
                self.link(x, next_x)
                x = next_x
            next_x = x.next
        return root
    def insert(self, key):
        if(None != self.search(self.root, key)):
            print("key have already in heap")
            return
        node = BinomialNode(key)
        self.root = self.combine(self.root, node)
        
    def reverse(self, root):
        #传入的root应该是二项树的根节点的child
        #此操作删掉根节点，并且将子树变成一个二项堆
        if(None == root):
            return None
        tail = None
        root.parent = None
        while(None != root.next):
            next = root.next
            root.next = tail
            tail = root
            root = next
            root.parent = None
        root.next = tail
        return root
    def remove(self, root, key):
        if(None == root):
            return None
        node = self.search(root, key)
        if(None == node):
            return root
        parent = node.parent
        while(None != parent):
            (node.key, parent.key) = (parent.key, node.key)
            node = parent
            parent = node.parent
        
        prev = None
        pos = root
        while(pos != node):
            prev = pos
            pos = pos.next
        if(None != prev):
            prev.next = node.next
        else:
            root = node.next
        return self.combine(root, self.reverse(node.child))
        
if __name__=='__main__':
    arr_a = [12,  7, 25, 15, 28, 33, 41]
    arr_b = [18, 35, 20, 42,  9, 31, 23,  6, 48, 11,24, 52, 13]
    ha = BinomialHeap()
    for i in arr_a:
        ha.insert(i)
    ha.dump()

B0  B1  B2  )的详细信息:
二项树B0:
    41(0) is root
二项树B1:
    28(1) is root
二项树B2:
    7(2) is root
